In [39]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import pandas as pd
from helper import *
from sklearn.preprocessing import LabelEncoder
pd.options.display.max_columns = None

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier

# MAKE SURE THE DATASETS ARE CORRECT

 ![caption](files/requirements.png)

## Import DataSet

In [40]:
sumdata_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AABABUTdx7YqCeBquA1Ky7z8a/The%20SUM%20dataset?dl=1#"
news_url  = "https://www.dropbox.com/sh/euppz607r6gsen2/AACq4aMWDOIw2I_SSGqJ-r2Oa/Online%20News%20Popularity%20(Mashable%20News)?dl=1"
housing_url = "https://www.dropbox.com/sh/euppz607r6gsen2/AAD6JGlvG5XADIjg9SCojvpya/House%20Sales%20in%20King%20County%2C%20USA?dl=1&preview=kc_house_data.csv"
all_urls = [sumdata_url,news_url,housing_url]

In [41]:
get_data(all_urls) # retrieves the data if there is NO data folder

In [42]:
sumdata_noise_path = "data/with noise/The SUM dataset, with noise.csv"
sumdata_path = "data/without noise/The SUM dataset, without noise.csv" 
housing_price_path ="data/kc_house_data.csv"
 

## Load datasets sum_noise

In [43]:
sumdata_noise = pd.read_csv(sumdata_noise_path, delimiter=";")

# Remove 'Instance' as it simply represents the row number
sumdata_noise = sumdata_noise.drop('Instance', axis = 1)
sumdata_noise.head(n=2)

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Noisy Target,Noisy Target Class
0,66957,74432,96087,103120,64272,150633,181787,180349,216912,304071,1434819,Very Large Number
1,96030,86875,108299,148025,16965,253819,258672,268851,404599,543092,2148748,Very Large Number


## Preprocess sum_noise dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [44]:
# Use 'Nosiy Target' as regression target
sumdata_noise_reg_Y = sumdata_noise['Noisy Target'].values.reshape(-1, 1)

# Use 'Nosiy Target Class' Large Number as regression target
sumdata_noise_classif_Y = pd.get_dummies(sumdata_noise['Noisy Target Class']).iloc[:, 0]
sumdata_noise_classif_Y = sumdata_noise_classif_Y.values.astype(int).reshape(-1,1)

# Use rest columns as explananatory variables
# We can simply use the same features for both as Noisy Target and Noisy Target Class are representing the samething
sumdata_noise_reg_X = sumdata_noise.iloc[:, 0:-2].values
sumdata_noise_classif_X = sumdata_noise.iloc[:, 0:-2].values

# Apply Feature Scaling for the classification variable
# As we are using KNN 
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
sumdata_noise_classif_X = scX.fit_transform(sumdata_noise_classif_X)


/Users/kiyim/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Load datasets sumdata

In [45]:
sumdata = pd.read_csv(sumdata_path, delimiter=";")

# Remove 'Instance' as it simply represents the row number
sumdata = sumdata.drop('Instance', axis = 1)
sumdata.head(n=2)

,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5 (meaningless),Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Target,Target Class
0,57326,68791,82549,99059,72624,142645,171174,205409,246491,295789,1073444,Very Large Number
1,87859,105431,126517,151820,19982,218621,262345,314814,377777,453332,1645184,Very Large Number


## Preprocess sumdata dataset

- Remove 'Instance' as it simply represents the row number
- Extract 'Nosiy Target' as regression target
- Extract 'Nosiy Class' as classification target
- Extract rest of the columns as explananatory variables
- Apply Feature Scaling to the dataset 

- Ensure all dataframe has been converted to numpy array


In [46]:
# Use 'Nosiy Target' as regression target
sumdata_reg_Y = sumdata['Target'].values.reshape(-1, 1)

# Use 'Nosiy Target Class' Large Number as classification target
sumdata_classif_Y = pd.get_dummies(sumdata['Target Class']).iloc[:, 0]
sumdata_classif_Y = sumdata_classif_Y.values.astype(int).reshape(-1,1)

# Use rest columns as explananatory variables
# We can simply use the same features for both as Target and Target Class are representing the same thing
sumdata_classif_X = sumdata.iloc[:, 0:-2].values
sumdata_reg_X = sumdata.iloc[:, 0:-2].values

# Apply Feature Scaling for the classification variable
# As we are using KNN 
from sklearn.preprocessing import StandardScaler
scX = StandardScaler()
scY = StandardScaler()
sumdata_classif_X = scX.fit_transform(sumdata_classif_X)



/Users/kiyim/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## News dataset

In [47]:
# online_news =  get_news_dataset()

news = get_news_dataset()
print(len(news))

39644


In [48]:
news.head(n=1)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.1,0.7,-0.35,-0.6,-0.2,0.5,-0.1875,0.0,0.1875,593


### Features  that will be selected BE CAREFUL ALL COLUMN NAMES HAVE SPACE AT THE START

In [49]:
columns= [ ' timedelta', ' n_tokens_title', ' n_tokens_content',
       ' n_unique_tokens', ' n_non_stop_words', ' n_non_stop_unique_tokens',
       ' num_hrefs', ' num_self_hrefs', ' num_imgs', ' num_videos',
       ' average_token_length', ' num_keywords', ' data_channel_is_lifestyle',
       ' data_channel_is_entertainment', ' data_channel_is_bus',
       ' data_channel_is_socmed', ' data_channel_is_tech',
       ' data_channel_is_world', ' kw_min_min', ' kw_max_min', ' kw_avg_min',
       ' kw_min_max', ' kw_max_max', ' kw_avg_max', ' kw_min_avg',
       ' kw_max_avg', ' kw_avg_avg', ' self_reference_min_shares',
       ' self_reference_max_shares', ' self_reference_avg_sharess', ' global_subjectivity', ' global_rate_positive_words',
       ' global_rate_negative_words', ' rate_positive_words',
       ' rate_negative_words', ' avg_positive_polarity',
       ' min_positive_polarity', ' max_positive_polarity',' title_subjectivity', ' abs_title_subjectivity',
       ' abs_title_sentiment_polarity', ' shares']

#### Classes
- 0 - 140550       class 1
- 140551 - 281101  class 2
- 281102 - 421651  class 3
- 421652 - 562202  class 4
- 562203 - 702752  class 5
- 702752 -         class 6

In [50]:
def create_classes(row):
    """

    """

  

    if(row[' shares'] <= 140550):
        
        return 1
    elif( row[' shares'] > 140550 and row[' shares'] <= 281101  ):
        
        return 2
    elif( row[' shares'] > 281101 and row[' shares'] <= 421651  ):
         
        return 3
    elif( row[' shares'] > 421651 and row[' shares'] <= 562202  ):
         
        return 4

    elif( row[' shares'] > 562202 and row[' shares'] <= 702752  ):
         
        return 5
    else:
        
        return 6

In [51]:
news = news[columns]
print(len(news))
# Classification  the target values are going t
news['label'] = news.apply(create_classes, axis=1)


39644


In [52]:
news_reg_Y = news[' shares'].values.reshape(-1, 1)

news_reg_X = news.iloc[:, 0:-2] # the last 2 columns are the target values 
news_classif_Y = news['label'].values.reshape(-1, 1)


# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()

# news_classif_Y = le.fit_transform(news_classif_Y)
max(news_classif_Y)

array([6])

# Fits Algorithms to datasets

In [53]:
data_chunks = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000,
1000000, 5000000, 10000000, 50000000, 100000000]

In [54]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from math import sqrt
def root_mean_square_error(y_actual, y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [55]:
def model( X, y, dataset_name, algorithm, isReg): 
    
    print ("Algorithm: {}\nDataset: {}\n".format( algorithm.__name__, dataset_name))
    for chunk in data_chunks:
        
        # if chunk is greater than the no. of examples quite from the chunking
        if chunk > X.shape[0]: 
            chunk = X.shape[0]
        
        print ("Chunk Size: {}\n".format(chunk))
        
        # generate the chunk file
        current_X = X[0:chunk]
        current_y = y[0:chunk]
        
        kFoldModelling(current_X, current_y, 10, algorithm, isReg)
        
        if chunk == X.shape[0]:
            break
              
         
    

In [56]:
def kFoldModelling (X, y, kfolds, algorithm, isReg):
    
    kf = KFold(n_splits=kfolds)
    rmse = np.zeros((10,1))
    mae = np.zeros((10,1))
    accuracy = np.zeros((10,1))
    precision = np.zeros((10,1))
    X = X.as_matrix()
    
    for i, (train_index, test_index) in enumerate(kf.split(X)):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # fit the model to the datset
        lm = algorithm()
        lm.fit(X_train, y_train)
        
        if isReg:     
            rmse[i] = root_mean_square_error(y_test, lm.predict(X_test))  # RMSE https://www.kaggle.com/wiki/RootMeanSquaredError
            mae[i] = mean_absolute_error(y_test, lm.predict(X_test))
        else:
            from sklearn.metrics import accuracy_score
            accuracy[i] = accuracy_score(y_test, lm.predict(X_test))
            precision[i] = precision_score(y_test, lm.predict(X_test))
        # print the result, we will need to have method that genereates the result csv file required.
    
    if isReg:     
        print ("Iteration: {}\nRMSE: {}\nMAE: {}\n".format( i, rmse.mean(), mae.mean()))
    else:
        print ("Iteration: {}\Accuracy: {}\nPrecision: {}\n".format( i, accuracy.mean(), precision.mean()))
    

## Fits Regression Algorithms to datasets

    - Linear Regression
    - Random Forest Regression
    

In [57]:
max(news_classif_Y)

array([6])

In [69]:
# model(sumdata_noise_reg_X, sumdata_noise_reg_Y, "The Sum Dataset(with noise)", LinearRegression, True)
# model(sumdata_reg_X, sumdata_reg_Y, "The Sum Dataset(without noise)",  LinearRegression, True)
# model(housing_price_reg_X, housing_price_reg_Y, "Housing Dataset",  LinearRegression, True)
# model(titanic_regression_X, titanic_regression_Y, "Titanic Dataset", LinearRegression, True)
#model(news_reg_X, news_reg_Y, "News dataset", LinearRegression, True)
# model(sumdata_noise_reg_X, sumdata_noise_reg_Y,"The Sum Dataset(with noise)",  RandomForestRegressor, True)
# model(sumdata_reg_X, sumdata_reg_Y, "The Sum Dataset(without noise)", RandomForestRegressor, True)
# model(housing_price_reg_X, housing_price_reg_Y, "Housing Dataset", RandomForestRegressor, True)
# model(titanic_regression_X, titanic_regression_Y, "Titanic Dataset", RandomForestRegressor, True)
 
from sklearn.linear_model import LogisticRegression
# model(sumdata_noise_classif_X, sumdata_noise_classif_Y, "The Sum Dataset(with noise)", LogisticRegression, False)
# model(sumdata_classif_X, sumdata_classif_Y, "The Sum Dataset(without noise)", LogisticRegression, False)
model(news_reg_X, news_classif_Y, "News dataset", LogisticRegression, False)
news_classif_Y.min()
# model(titanic_classification_X, titanic_classification_y, "Titanic Dataset", LogisticRegression, False)

# from sklearn.svm import SVC
# model(sumdata_noise_classi_X, sumdata_noise_classif_Y, "The Sum Dataset(with noise)", SVC, False)
# model(sumdata_classi_X, sumdata_classif_Y, "The Sum Dataset(without noise)", SVC, False)
# model(housing_price_classi_X, housing_price_classif_Y, "Housing Dataset", SVC, False)
# model(titanic_classification_X, titanic_classification_y, "Titanic Dataset", SVC, False)


1